In [1]:
import os
os.chdir('../../scripts/external-data-download/')

In [2]:
OUTPUT_RELATIVE_PATH = "../../data/"
def create_path():
    """
        
    """
    path = ["external-raw-data", "curated/external-data"]
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
    print('Already Create Paths')

create_path()

Already Create Paths


In [3]:
from download import download

download.download_all()